In [1]:
import glob
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
files = glob.glob('../vz-hackathon/**')

In [4]:
parking_violations = [f for f in files if 'Parking_Violations_in' in f]

In [5]:
list_ = []
for file_ in parking_violations[:]:
    df = pd.read_csv(file_,index_col=None, header=0)
    
    filename = file_[len('../vz-hackathon/'):]
    
    df['filename'] = filename
       
    list_.append(df)
    
frame = pd.concat(list_)

# TESTs
assert frame.filename.nunique() == len(parking_violations)
frame.columns = [col.lower() for col in frame.columns]
frame = frame.reset_index(drop=True)

In [6]:
df = frame.copy()

In [7]:
# fuctnions

def mil_to_time(x):
   
    if x == 'nan':
        return '00:00:00.000Z'
        
    x = x.split('.')[0]
    lg = len(x)
    
    if lg == 4:
        t = x[:2] + ':' + x[2:] + ':00.000Z'
      
    elif lg == 3:
        t = '0' + x[0] + ':' + x[1:] + ':00.000Z'
    
    elif lg == 2:
        t = '0' + '0' + ':' + x + ':00.000Z'
    
    elif lg == 1:
        t = '0' + '0' + ':' + '0' + x + ':00.000Z'
    
    else:
        t = '00:00.000Z'
        
    if int(t[3]) > 5:
        t = t[:2]+ ':' + '5' + t[4:]
        
    return t

In [8]:
## Feature engineering
df['issue_time_military'] = df.issue_time.apply(str).apply(mil_to_time)
dates = df.ticket_issue_date.str[:10] + 'T' #+ 
df['ticket_issue_datetime'] = dates + df.issue_time_military
df['holiday'] = df.holiday != 0


In [9]:
# ## Del redudant columns
del df['day_of_week']
del df['month_of_year']
del df['week_of_year']
del df['issue_time']
del df['issue_time_military']
del df['ticket_issue_date']

In [10]:
## Testing datetime format
# for i, t in enumerate(df.ticket_issue_datetime):
#     try:
#         pd.to_datetime(t)
#     except:
#         print i, t

In [13]:
df.head(2)

,﻿x,y,objectid,rowid_,holiday,violation_code,violation_description,location,rp_plate_state,body_style,address_id,streetsegid,xcoord,ycoord,filename,ticket_issue_datetime
0,-77.028867,38.895245,85866,10692752,False,P001,PARK ABREAST OF ANOTHER VEHICLE,1300 BLOCK PENNSYLVANIA AVE NW NOR,MD,TK,801823,1681.0,397496,136377,Parking_Violations_in_April_2009.csv,2009-04-28T14:16:00.000Z
1,-77.043465,38.903141,85918,10692911,False,P159,NO STOPPING OR STANDING IN AM RUSH HOUR ZONE,1100 BLOCK 19TH ST NW WEST SIDE,VA,PU,814327,12343.0,396230,137254,Parking_Violations_in_April_2009.csv,2009-04-23T08:31:00.000Z


In [15]:
# frame.to_json('parking_violations.json')
df.to_csv('parking_violations.csv', sep='\t', index=False)


In [16]:
df.sample(10000).to_csv('parking_violations_sample.csv', sep='\t', index=False)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12189654 entries, 0 to 12189653
Data columns (total 16 columns):
﻿x                       float64
y                        float64
objectid                 int64
rowid_                   int64
holiday                  bool
violation_code           object
violation_description    object
location                 object
rp_plate_state           object
body_style               object
address_id               int64
streetsegid              float64
xcoord                   int64
ycoord                   int64
filename                 object
ticket_issue_datetime    object
dtypes: bool(1), float64(3), int64(5), object(7)
memory usage: 1.4+ GB
